In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
sub = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv")
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
persuade = pd.read_csv("/kaggle/input/persuade2-0/persuade_2.0_human_scores_demo_id_github.csv")
print("Successfully loaded!")

Successfully loaded!


In [3]:
gpt3bucuk = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
gpt4 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv')
falcon180b = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/falcon_180b_v1.csv")
llama_70b_v1 = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_70b_v1.csv")
llama_70b_v2 = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_70b_v2.csv")
llama_falcon_v3 = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_falcon_v3.csv")
anthrop_c = pd.read_csv("/kaggle/input/hello-claude-1000-essays-from-anthropic/persuade15_claude_instant1.csv")
palm = pd.read_csv("/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv")
print("Successful")

Successful


In [4]:
falcon180b.rename(columns={'generated_text': 'text'}, inplace=True)
llama_70b_v1.rename(columns={'generated_text': 'text'}, inplace=True)
llama_70b_v2.rename(columns={'generated_text': 'text'}, inplace=True)
anthrop_c.rename(columns={'essay_text': 'text'}, inplace=True)
persuade.rename(columns={'full_text': 'text'}, inplace=True)
print("Successful")

Successful


In [5]:
persuade['generated'] = 0
falcon180b['generated'] = 1
llama_70b_v1['generated'] = 1
anthrop_c['generated'] = 1
palm['generated'] = 1
print("Successful")

Successful


# **Müdahale öncesi her metinde analiz edilmesi gerekenler** <br>
- Paragraf başına kelime sayısı✅
- Ortalama Paragraf sayısı✅
- Ortalama paragraf uzunluğu✅
- Ortalama Cümle sayısı✅
- Toplam kelime sayısı (Stopwords'süz)✅
- Metindeki toplam Benzersiz karakter sayısı (Belki rakam sayıları ayrıca sayılabilir belki) ❌ Çıkardım.
- Toplam #$£½{[] şu tarz özel karakterlerin sayısı ❌ (Ayrıca .,!? gibi işaret sayıları) ✅
- Ortalama kelime uzunluğu ✅ [NLP ile daha iyi olabilir...]
- Ortalama cümle uzunluğu ✅ [NLP ile daha iyi olabilir...]
- Cümlelerin karmaşıklığı tüm metinde (bir cümledeki kelime sayısı ve bağlaç kullanımı sayıları) ✅
- Cümlelerdeki farklı kelime sayısı, bu sayede hep aynı şeyden mi bahsetti anlayacağız. [NLP ile daha iyi olabilir...]
- Belirli kelimelerin veya kelime gruplarının metindeki frekansları. (Bunu GPT'ye sor.) [NLP ile daha iyi olabilir...]
- Belirli bağlaçları içeren cümlelerin sayısı (yani her bağlaçın kendine özel tespiti and, as, or vb. kaç tane geçmiş) ❌ Bence işe yaramayacak. Tekrar düşünülebilir elbette.
- Yinelemelerin sıklığı. (Bunu da NLP'de ölçeriz.)<br> 
Not: (Dolar işareti koyduğun zaman ingilizcede bulunmayan karakterleri kırmızı olarak gösteriyor.)

In [6]:
# Her hücredeki metinlerden '\n\n' ifadesi sayısını bulan fonksiyon
def count_newlines(text):
    return text.count('\n\n')

# Her hücredeki metinlerden toplam kelime sayısını bulan fonksiyon
def count_words(text):
    return len(text.split())

# Her hücredeki metinlerden '\n\n' ifadesi sayısına göre toplam kelime sayısını hesapla ve böl LLM
gpt4['word_per_paragraph'] = gpt4['text'].apply(count_words) / (gpt4['text'].apply(count_newlines) + 1)
gpt3bucuk['word_per_paragraph'] = gpt3bucuk['text'].apply(count_words) / (gpt3bucuk['text'].apply(count_newlines) + 1)
falcon180b['word_per_paragraph'] = falcon180b['text'].apply(count_words) / (falcon180b['text'].apply(count_newlines) + 1)
llama_70b_v1['word_per_paragraph'] = llama_70b_v1['text'].apply(count_words) / (llama_70b_v1['text'].apply(count_newlines) + 1)
llama_70b_v2['word_per_paragraph'] = llama_70b_v2['text'].apply(count_words) / (llama_70b_v2['text'].apply(count_newlines) + 1)
llama_falcon_v3['word_per_paragraph'] = llama_falcon_v3['text'].apply(count_words) / (llama_falcon_v3['text'].apply(count_newlines) + 1)
anthrop_c['word_per_paragraph'] = anthrop_c['text'].apply(count_words) / (anthrop_c['text'].apply(count_newlines) + 1)
palm['word_per_paragraph'] = palm['text'].apply(count_words) / (palm['text'].apply(count_newlines) + 1)

# insan yazısı
test_essays['word_per_paragraph'] = test_essays['text'].apply(count_words) / (test_essays['text'].apply(count_newlines) + 1)
train_essays['word_per_paragraph'] = train_essays['text'].apply(count_words) / (train_essays['text'].apply(count_newlines) + 1)
persuade['word_per_paragraph'] = persuade['text'].apply(count_words) / (persuade['text'].apply(count_newlines) + 1)
print('Küçük bir hatırlatma: " " ifadesi yanında herhangi bir obje olursa sayılıyor. "" = 1 sayılıyor. "s = 1 sayılıyor. "smart phone" = 2')
print("Successful")

Küçük bir hatırlatma: " " ifadesi yanında herhangi bir obje olursa sayılıyor. "" = 1 sayılıyor. "s = 1 sayılıyor. "smart phone" = 2
Successful


In [7]:
# TÜM METİNLERDEKİ PARAGRAF SAYISI, doğru şekilde çalışıyor. \n\n + 1 = paragraph_counts, doğru yani.
def paragraf_sayisi(metin):
    paragraflar = metin.split('\n\n')
    return len(paragraflar)


gpt4['paragraph_counts'] = gpt4['text'].apply(paragraf_sayisi)
gpt3bucuk['paragraph_counts'] = gpt3bucuk['text'].apply(paragraf_sayisi)
falcon180b['paragraph_counts'] = falcon180b['text'].apply(paragraf_sayisi)
llama_70b_v1['paragraph_counts'] = llama_70b_v1['text'].apply(paragraf_sayisi)
llama_70b_v2['paragraph_counts'] = llama_70b_v2['text'].apply(paragraf_sayisi)
llama_falcon_v3['paragraph_counts'] = llama_falcon_v3['text'].apply(paragraf_sayisi)
anthrop_c['paragraph_counts'] = anthrop_c['text'].apply(paragraf_sayisi)
palm['paragraph_counts'] = palm['text'].apply(paragraf_sayisi)

test_essays['paragraph_counts'] = test_essays['text'].apply(paragraf_sayisi)
train_essays['paragraph_counts'] = train_essays['text'].apply(paragraf_sayisi)
persuade['paragraph_counts'] = persuade['text'].apply(paragraf_sayisi)
print("Successful")

Successful


In [8]:
# tüm paragrafların uzunluğunu sayıyor. Aslında kelime saymasıyla aynı şeydi ama sen bilirsin.
def newline_count_and_last_paragraph_length(metin):
    paragraflar = metin.split('\n\n')
    newline_count = len(paragraflar) - 1  # Son paragrafın sonunda \n\n olmayabilir
    last_paragraph_length = len(paragraflar[-1]) if newline_count > 0 else len(metin)
    return newline_count, last_paragraph_length

# Ortalama paragraf uzunluğunu hesaplamak için bir fonksiyon tanımlayalım
def ortalama_paragraf_uzunlugu(metin):
    newline_count, last_paragraph_length = newline_count_and_last_paragraph_length(metin)
    paragraf_sayisi = newline_count + 1
    paragraflar = metin.split('\n\n')  # Paragrafları tekrar ayarla
    karakter_sayisi = len("".join(paragraflar))  # \n\n ifadelerini hesaplamada görmezden gel
    return karakter_sayisi / paragraf_sayisi

gpt4['average_paragraph_length'] = gpt4['text'].apply(ortalama_paragraf_uzunlugu)
gpt3bucuk['average_paragraph_length'] = gpt3bucuk['text'].apply(ortalama_paragraf_uzunlugu)
falcon180b['average_paragraph_length'] = falcon180b['text'].apply(ortalama_paragraf_uzunlugu)
llama_70b_v1['average_paragraph_length'] = llama_70b_v1['text'].apply(ortalama_paragraf_uzunlugu)
llama_70b_v2['average_paragraph_length'] = llama_70b_v2['text'].apply(ortalama_paragraf_uzunlugu)
llama_falcon_v3['average_paragraph_length'] = llama_falcon_v3['text'].apply(ortalama_paragraf_uzunlugu)
anthrop_c['average_paragraph_length'] = anthrop_c['text'].apply(ortalama_paragraf_uzunlugu)
palm['average_paragraph_length'] = palm['text'].apply(ortalama_paragraf_uzunlugu)

test_essays['average_paragraph_length'] = test_essays['text'].apply(ortalama_paragraf_uzunlugu)
train_essays['average_paragraph_length'] = train_essays['text'].apply(ortalama_paragraf_uzunlugu)
persuade['average_paragraph_length'] = persuade['text'].apply(ortalama_paragraf_uzunlugu)
print("Successful")

Successful


In [9]:
# METİNDEKİ TOPLAM CÜMLE SAYISI
def ortalama_cumle_sayisi(metin):
    cumleler = re.split(r'[.!?]+|\.{3}|:', metin)
    cumle_sayisi = len([cumle for cumle in cumleler if cumle.strip() != ''])
    return cumle_sayisi

gpt4['sentence_count'] = gpt4['text'].apply(ortalama_cumle_sayisi)
gpt3bucuk['sentence_count'] = gpt3bucuk['text'].apply(ortalama_cumle_sayisi)
falcon180b['sentence_count'] = falcon180b['text'].apply(ortalama_cumle_sayisi)
llama_70b_v1['sentence_count'] = llama_70b_v1['text'].apply(ortalama_cumle_sayisi)
llama_70b_v2['sentence_count'] = llama_70b_v2['text'].apply(ortalama_cumle_sayisi)
llama_falcon_v3['sentence_count'] = llama_falcon_v3['text'].apply(ortalama_cumle_sayisi)
anthrop_c['sentence_count'] = anthrop_c['text'].apply(ortalama_cumle_sayisi)
palm['sentence_count'] = palm['text'].apply(ortalama_cumle_sayisi)

test_essays['sentence_count'] = test_essays['text'].apply(ortalama_cumle_sayisi)
train_essays['sentence_count'] = train_essays['text'].apply(ortalama_cumle_sayisi)
persuade['sentence_count'] = persuade['text'].apply(ortalama_cumle_sayisi)
print("Successful")

Successful


In [10]:
# Toplam kelime sayısını bulmak için bir fonksiyon tanımlayalım
def toplam_kelime_sayisi(metin):
    # "\n\n" ifadesini boşluk karakteriyle değiştir
    metin = metin.replace('\n\n', ' ')
    # Noktalama işaretleri ve özel karakterleri temizle
    metin = re.sub(r'[^\w\s]', '', metin)
    # Boşluklara göre metni bölelim ve boş olmayanları seçelim
    kelimeler = [kelime for kelime in metin.split() if kelime.strip() != '']
    return len(kelimeler)

# DataFrame'e yeni bir sütun ekleyerek toplam kelime sayısını hesaplayalım
gpt4['total_word_count'] = gpt4['text'].apply(toplam_kelime_sayisi)
gpt3bucuk['total_word_count'] = gpt3bucuk['text'].apply(toplam_kelime_sayisi)
falcon180b['total_word_count'] = falcon180b['text'].apply(toplam_kelime_sayisi)
llama_70b_v1['total_word_count'] = llama_70b_v1['text'].apply(toplam_kelime_sayisi)
llama_70b_v2['total_word_count'] = llama_70b_v2['text'].apply(toplam_kelime_sayisi)
llama_falcon_v3['total_word_count'] = llama_falcon_v3['text'].apply(toplam_kelime_sayisi)
anthrop_c['total_word_count'] = anthrop_c['text'].apply(toplam_kelime_sayisi)
palm['total_word_count'] = palm['text'].apply(toplam_kelime_sayisi)

test_essays['total_word_count'] = test_essays['text'].apply(toplam_kelime_sayisi)
train_essays['total_word_count'] = train_essays['text'].apply(toplam_kelime_sayisi)
persuade['total_word_count'] = persuade['text'].apply(toplam_kelime_sayisi)
print("Successful")

Successful


In [11]:
# Soru işareti, üç nokta, ünlem ve nokta sayısını hesaplamak için bir fonksiyon tanımlayalım
def sayac(metin):
    soru_isareti_sayisi = metin.count('?')
    uc_nokta_sayisi = metin.count('...')
    unlem_sayisi = metin.count('!')
    
    # Üç noktalı ifadelerin içerisindeki üç noktaları saymamak için düzeltme
    nokta_sayisi = metin.count('.') - 3 * uc_nokta_sayisi
    
    return nokta_sayisi, uc_nokta_sayisi, unlem_sayisi, soru_isareti_sayisi

# DataFrame'e yeni bir sütun ekleyerek sayıları hesaplayalım
gpt4[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = gpt4['text'].apply(sayac).apply(pd.Series)
gpt3bucuk[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = gpt3bucuk['text'].apply(sayac).apply(pd.Series)
falcon180b[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = falcon180b['text'].apply(sayac).apply(pd.Series)
llama_70b_v1[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = llama_70b_v1['text'].apply(sayac).apply(pd.Series)
llama_70b_v2[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = llama_70b_v2['text'].apply(sayac).apply(pd.Series)
llama_falcon_v3[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = llama_falcon_v3['text'].apply(sayac).apply(pd.Series)
anthrop_c[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = anthrop_c['text'].apply(sayac).apply(pd.Series)
palm[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = palm['text'].apply(sayac).apply(pd.Series)

test_essays[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = test_essays['text'].apply(sayac).apply(pd.Series)
train_essays[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = train_essays['text'].apply(sayac).apply(pd.Series)
persuade[['point_count', 'three_point_count', 'exclamation_count', 'question_mark_count']] = persuade['text'].apply(sayac).apply(pd.Series)
print("Successful")

Successful


In [12]:
# Metindeki ortalama kelime uzunluğunu hesaplamak için bir fonksiyon tanımlayalım
def ortalama_kelime_uzunlugu(metin):
    kelimeler = re.findall(r'\b\w+\b', metin)  # Boşluklarla ayrılmış kelimeleri bul
    if not kelimeler:
        return 0  # Metin boşsa ortalama kelime uzunluğu 0
    toplam_kelime_uzunlugu = sum(len(kelime) for kelime in kelimeler)
    return toplam_kelime_uzunlugu / len(kelimeler)

# DataFrame'e yeni bir sütun ekleyerek ortalama kelime uzunluğunu hesaplayalım
gpt4['average_word_length'] = gpt4['text'].apply(ortalama_kelime_uzunlugu)
gpt3bucuk['average_word_length'] = gpt3bucuk['text'].apply(ortalama_kelime_uzunlugu)
falcon180b['average_word_length'] = falcon180b['text'].apply(ortalama_kelime_uzunlugu)
llama_70b_v1['average_word_length'] = llama_70b_v1['text'].apply(ortalama_kelime_uzunlugu)
llama_70b_v2['average_word_length'] = llama_70b_v2['text'].apply(ortalama_kelime_uzunlugu)
llama_falcon_v3['average_word_length'] = llama_falcon_v3['text'].apply(ortalama_kelime_uzunlugu)
anthrop_c['average_word_length'] = anthrop_c['text'].apply(ortalama_kelime_uzunlugu)
palm['average_word_length'] = palm['text'].apply(ortalama_kelime_uzunlugu)

test_essays['average_word_length'] = test_essays['text'].apply(ortalama_kelime_uzunlugu)
train_essays['average_word_length'] = train_essays['text'].apply(ortalama_kelime_uzunlugu)
persuade['average_word_length'] = persuade['text'].apply(ortalama_kelime_uzunlugu)
print("Successful")

Successful


In [13]:
# Metindeki ortalama cümle uzunluğunu hesaplamak için bir fonksiyon tanımlayalım
def ortalama_cumle_uzunlugu(metin):
    cumleler = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', metin)
    if not cumleler:
        return 0  # Metin boşsa ortalama cümle uzunluğu 0
    toplam_cumle_uzunlugu = sum(len(re.findall(r'\b\w+\b', cumle)) for cumle in cumleler)
    return toplam_cumle_uzunlugu / len(cumleler)

# DataFrame'e yeni bir sütun ekleyerek ortalama cümle uzunluğunu hesaplayalım
gpt4['average_sentence_length'] = gpt4['text'].apply(ortalama_cumle_uzunlugu)
gpt3bucuk['average_sentence_length'] = gpt3bucuk['text'].apply(ortalama_cumle_uzunlugu)
falcon180b['average_sentence_length'] = falcon180b['text'].apply(ortalama_cumle_uzunlugu)
llama_70b_v1['average_sentence_length'] = llama_70b_v1['text'].apply(ortalama_cumle_uzunlugu)
llama_70b_v2['average_sentence_length'] = llama_70b_v2['text'].apply(ortalama_cumle_uzunlugu)
llama_falcon_v3['average_sentence_length'] = llama_falcon_v3['text'].apply(ortalama_cumle_uzunlugu)
anthrop_c['average_sentence_length'] = anthrop_c['text'].apply(ortalama_cumle_uzunlugu)
palm['average_sentence_length'] = palm['text'].apply(ortalama_cumle_uzunlugu)

test_essays['average_sentence_length'] = test_essays['text'].apply(ortalama_cumle_uzunlugu)
train_essays['average_sentence_length'] = train_essays['text'].apply(ortalama_cumle_uzunlugu)
persuade['average_sentence_length'] = persuade['text'].apply(ortalama_cumle_uzunlugu)
print("Successful")

Successful


In [14]:
# Cümlelerdeki benzersiz kelime ve İngilizce bağlaç sayısını hesaplamak için bir fonksiyon tanımlayalım
def cümle_kompleksligi(metin):
    kelimeler = re.findall(r'\b\w+\b', metin.lower())  # Boşluklarla ayrılmış kelimeleri bul, küçük harfe çevir
    benzersiz_kelimeler = set(kelimeler)
    ingilizce_baglac_sayisi = len(re.findall(
        r'\b(?:and|or|but|although|while|because|since|unless|until|if|as|when|where|after|before|by|for|while|since)\b', 
        metin.lower()))
    
    return len(benzersiz_kelimeler), ingilizce_baglac_sayisi

# DataFrame'e yeni sütunlar ekleyerek benzersiz kelime ve bağlaç sayılarını hesaplayalım
gpt4[['unique_word_in_all_text', 'conjunction_count']] = gpt4['text'].apply(cümle_kompleksligi).apply(pd.Series)
gpt3bucuk[['unique_word_in_all_text', 'conjunction_count']] = gpt3bucuk['text'].apply(cümle_kompleksligi).apply(pd.Series)
falcon180b[['unique_word_in_all_text', 'conjunction_count']] = falcon180b['text'].apply(cümle_kompleksligi).apply(pd.Series)
llama_70b_v1[['unique_word_in_all_text', 'conjunction_count']] = llama_70b_v1['text'].apply(cümle_kompleksligi).apply(pd.Series)
llama_70b_v2[['unique_word_in_all_text', 'conjunction_count']] = llama_70b_v2['text'].apply(cümle_kompleksligi).apply(pd.Series)
llama_falcon_v3[['unique_word_in_all_text', 'conjunction_count']] = llama_falcon_v3['text'].apply(cümle_kompleksligi).apply(pd.Series)
anthrop_c[['unique_word_in_all_text', 'conjunction_count']] = anthrop_c['text'].apply(cümle_kompleksligi).apply(pd.Series)
palm[['unique_word_in_all_text', 'conjunction_count']] = palm['text'].apply(cümle_kompleksligi).apply(pd.Series)

test_essays[['unique_word_in_all_text', 'conjunction_count']] = test_essays['text'].apply(cümle_kompleksligi).apply(pd.Series)
train_essays[['unique_word_in_all_text', 'conjunction_count']] = train_essays['text'].apply(cümle_kompleksligi).apply(pd.Series)
persuade[['unique_word_in_all_text', 'conjunction_count']] = persuade['text'].apply(cümle_kompleksligi).apply(pd.Series)
print("Successful")

Successful


# (En son birleştirme işlemim) = 3 satır

In [15]:
baskanlik_secimi = train_essays[train_essays['prompt_id'] == 1]
cars = train_essays[train_essays['prompt_id'] == 0]
persuade_car_free_cities = persuade[persuade['prompt_name'] == 'Car-free cities']
persuade_electoral = persuade[persuade['prompt_name'] == 'Does the electoral college work?']
persuade_venus = persuade[persuade['prompt_name'] == 'Exploring Venus']
persuade_mars = persuade[persuade['prompt_name'] == "The Face on Mars"]
persuade_facial = persuade[persuade['prompt_name'] == "Facial action coding system"]
persuade_cowboy = persuade[persuade['prompt_name'] == '"A Cowboy Who Rode the Waves"']
persuade_driverless = persuade[persuade['prompt_name'] == 'Driverless cars']
print("Successful")

Successful


In [16]:
# Bir liste oluşturun ve DataFrame'leri bu listeye ekleyin
dataframes = [baskanlik_secimi,
              cars,
              persuade_car_free_cities,
              persuade_electoral,
              persuade_venus,
              persuade_mars,
              persuade_facial,
              persuade_cowboy,
              persuade_driverless,
              gpt4,
              gpt3bucuk,
              falcon180b,
              llama_70b_v1,
              llama_70b_v2,
              llama_falcon_v3,
              anthrop_c,
              palm,
             ]

# "text" ve "generated" sütunlarını içeren yeni bir DataFrame oluşturun
one_piece = pd.concat([df[['text', 'generated','word_per_paragraph','paragraph_counts', 'average_paragraph_length', 'sentence_count',
       'total_word_count', 'point_count', 'three_point_count',
       'exclamation_count', 'question_mark_count', 'average_word_length',
       'average_sentence_length', 'unique_word_in_all_text',
       'conjunction_count']] for df in dataframes], ignore_index=True)
print("Successful")

Successful


In [17]:
one_piece

,text,generated,word_per_paragraph,paragraph_counts,average_paragraph_length,sentence_count,total_word_count,point_count,three_point_count,exclamation_count,question_mark_count,average_word_length,average_sentence_length,unique_word_in_all_text,conjunction_count
0,The electrol college system is an unfair syste...,0,116.400000,5,660.600000,22,580,21,0,0,1,4.565812,26.590909,196,56
1,"Dear state senator, It is the utmost respect t...",0,72.000000,6,423.500000,26,431,24,0,0,2,4.747126,16.730769,192,27
2,"""It's official: The electoral college is unfai...",0,94.200000,5,539.800000,30,471,24,3,1,4,4.533613,19.833333,197,36
3,The Electoral College has been kept for centur...,0,113.800000,5,656.400000,24,569,24,0,0,0,4.661996,23.791667,231,42
4,"Dear senator, Retain the Electoral College. Th...",0,88.500000,4,538.250000,22,354,22,0,0,0,4.741935,31.000000,165,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27731,"Dear Senator,\n\nI am writing to you today to ...",1,34.727273,11,201.727273,23,382,22,0,0,0,4.632391,18.523810,173,30
27732,"Dear Senator,\n\nI am writing to you today to ...",1,31.909091,11,189.909091,25,351,24,0,0,0,4.829545,14.666667,162,18
27733,"Dear Senator,\n\nI am writing to you today to ...",1,34.375000,8,198.125000,17,275,16,0,0,0,4.672727,18.333333,134,19
27734,"Dear Senator,\n\nI am writing to you today to ...",1,31.750000,8,180.000000,16,254,15,0,0,0,4.594488,15.875000,120,18


# İlk model, Ya Allah, Bismillah

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# X ve y'yi seçiyorum
y = one_piece['generated']  # 'generated' sütunu y değeri
X = one_piece.drop(['text', 'generated'], axis=1)  # 'text' ve 'generated' sütunlarını çıkart

# Veri setini eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))

# Modeli eğitin
model.fit(X_train, y_train)

# Eğitim veri seti üzerinde tahmin yapın
train_predictions = model.predict_proba(X_train)[:, 1]

# Test veri seti üzerinde tahmin yapın
test_predictions = model.predict_proba(X_test)[:, 1]

# ROC-AUC skorlarını değerlendirin
train_roc_auc = roc_auc_score(y_train, train_predictions)
test_roc_auc = roc_auc_score(y_test, test_predictions)

print(f"Train ROC-AUC Score: {train_roc_auc}")
print(f"Test ROC-AUC Score: {test_roc_auc}")

Train ROC-AUC Score: 0.959149038101703
Test ROC-AUC Score: 0.9565567603106712


In [19]:
# Test seti üzerinde tahmin yapın
y_pred = model.predict(X_test)

In [20]:
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Doğruluk (accuracy)
accuracy = accuracy_score(y_test, y_pred)

# Hassasiyet (precision)
precision = precision_score(y_test, y_pred)

# Duyarlılık (recall)
recall = recall_score(y_test, y_pred)

# F1 Skoru
f1 = f1_score(y_test, y_pred)

# ROC-AUC Skoru (sadece ikili sınıflandırma modelleri için)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print("ROC:", roc_auc)
print("f1:", f1)
print("recall:", recall)
print("precision:", precision)
print("accuracy:", accuracy)

ROC: 0.9565567603106712
f1: 0.8921239268383726
recall: 0.8874860750092833
precision: 0.8968105065666041
accuracy: 0.8958183129055516


In [22]:
from sklearn.metrics import mean_squared_error, r2_score

# Modelin tahminleri
test_predictions = model.predict(X_test)

# Regresyon metriklerini kullanma
mse = mean_squared_error(y_test, test_predictions)
r2 = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 0.10418168709444844
R2 Score: 0.5829176375411896


### Modele kendin tahmin yaptırmak istersen.

In [23]:
# Manuel olarak tahminler yapın
manual_X = pd.DataFrame({'word_per_paragraph': [100],
                         'paragraph_counts': [100], 
                         'average_paragraph_length': [100], 
                         'sentence_count': [100],
                         'total_word_count': [6], 
                         'point_count': [100], 
                         'three_point_count': [6],
                         'exclamation_count': [5000],
                         'question_mark_count': [6],
                         'average_word_length': [6],
                         'average_sentence_length': [100],
                         'unique_word_in_all_text': [6],
                         'conjunction_count': [6]
})

# predict metodu kullanarak sınıflandırma tahminleri yapma
predicted_class = model.predict(manual_X)

# predict_proba metodu kullanarak sınıf olasılıklarını alma (sadece sınıflandırma modellerinde)
class_probabilities = model.predict_proba(manual_X)

print("Predicted Class:", predicted_class)
print("Class Probabilities:", class_probabilities)

Predicted Class: [1]
Class Probabilities: [[0. 1.]]


# Test verisine uygulama ve submission

In [24]:
z = test_essays[['id']].copy()

In [25]:
# 'generated' sütununu ekleyin ve rastgele boş sayılarla doldurun
z['generated'] = np.random.rand(len(z))

In [26]:
z

,id,generated
0,0000aaaa,0.010756
1,1111bbbb,0.352186
2,2222cccc,0.965760


In [27]:
# Test veri setini yükleyin veya oluşturun
# X_test'i tanımlayın

# Model ile tahmin yapın
predictions = model.predict_proba(X_test)[:, 1]

In [28]:
y

0        0
1        0
2        0
3        0
4        0
        ..
27731    1
27732    1
27733    1
27734    1
27735    1
Name: generated, Length: 27736, dtype: int64

In [29]:
sub

,id,generated
0,0000aaaa,0.1
1,1111bbbb,0.9
2,2222cccc,0.4


In [30]:
X

,word_per_paragraph,paragraph_counts,average_paragraph_length,sentence_count,total_word_count,point_count,three_point_count,exclamation_count,question_mark_count,average_word_length,average_sentence_length,unique_word_in_all_text,conjunction_count
0,116.400000,5,660.600000,22,580,21,0,0,1,4.565812,26.590909,196,56
1,72.000000,6,423.500000,26,431,24,0,0,2,4.747126,16.730769,192,27
2,94.200000,5,539.800000,30,471,24,3,1,4,4.533613,19.833333,197,36
3,113.800000,5,656.400000,24,569,24,0,0,0,4.661996,23.791667,231,42
4,88.500000,4,538.250000,22,354,22,0,0,0,4.741935,31.000000,165,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27731,34.727273,11,201.727273,23,382,22,0,0,0,4.632391,18.523810,173,30
27732,31.909091,11,189.909091,25,351,24,0,0,0,4.829545,14.666667,162,18
27733,34.375000,8,198.125000,17,275,16,0,0,0,4.672727,18.333333,134,19
27734,31.750000,8,180.000000,16,254,15,0,0,0,4.594488,15.875000,120,18


# Beynim yandı dur

In [31]:
# final_test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
# final_test
z = test_essays.drop(['text', 'prompt_id','id',], axis=1)  # 'text' ve 'generated' sütunlarını çıkart

In [32]:
z

,word_per_paragraph,paragraph_counts,average_paragraph_length,sentence_count,total_word_count,point_count,three_point_count,exclamation_count,question_mark_count,average_word_length,average_sentence_length,unique_word_in_all_text,conjunction_count
0,3.0,1,12.0,1,3,1,0,0,0,3.0,3.0,3,0
1,3.0,1,12.0,1,3,1,0,0,0,3.0,3.0,3,0
2,3.0,1,12.0,1,3,1,0,0,0,3.0,3.0,3,0


In [33]:
pred = model.predict_proba(z)[:, 1]
pred

array([0.03084638, 0.03084638, 0.03084638])

In [34]:
pred

array([0.03084638, 0.03084638, 0.03084638])

In [35]:
# test_result = pred[0]
test_result = pred[:]
test_result

array([0.03084638, 0.03084638, 0.03084638])

In [36]:
final_submission = pd.DataFrame(test_essays['id'])
final_submission['generated'] = test_result
final_submission

,id,generated
0,0000aaaa,0.030846
1,1111bbbb,0.030846
2,2222cccc,0.030846


In [37]:
final_submission.to_csv('submission.csv', index=False)

In [38]:
messi = pd.read_csv("/kaggle/working/submission.csv")
messi

,id,generated
0,0000aaaa,0.030846
1,1111bbbb,0.030846
2,2222cccc,0.030846
